<a href="https://colab.research.google.com/github/lorenzofilitti/Corpus-Material/blob/main/Llama_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import requests

# NOTE: ollama must be running for this to work, start the ollama app or run `ollama serve`
model = "llama3.1"  # TODO: update this for whatever model you wish to use

file = open('Recensioni.json',  encoding='utf-8')
data = json.load(file)

prompt = "con una sola parola descrivi i testi che ti vengono forniti"





def chat(messages):
    r = requests.post(
        "http://localhost:49710/api/chat",
        json={"model": model, "messages": messages, "stream": True},
	stream=True
    )
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        body = json.loads(line)
        if "error" in body:
            raise Exception(body["error"])
        if body.get("done") is False:
            message = body.get("message", "")
            content = message.get("content", "")
            output += content
            # the response streams one token at a time, print that as we receive it
            print(content, end="", flush=True)

        if body.get("done", False):
            message["content"] = output
            return message

messages = []
messages.append({"role": "user", "content": prompt})
message = chat(messages)


for prodotti in data["Prodotti"]:
    for recensioni in prodotti["recensioni"]:
        messages = []
        recensione = recensioni["testo"]
        user = recensioni["utente"]
        print(user, "\n", recensione, "\n")
        messages.append({"role": "user", "content": prompt})
        messages.append({"role": "user", "content": recensione})
        message = chat(messages)
        print("\n")



file.close()



ConnectionError: HTTPConnectionPool(host='localhost', port=49710): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b33ff2b60b0>: Failed to establish a new connection: [Errno 111] Connection refused'))